In [1]:
#import packages
import requests
from bs4 import BeautifulSoup
import re
import json
import os
import sqlite3 as sql
import numpy as np
import pandas as pd
from lxml import html
from selenium import webdriver



C:\Users\Rani\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# initializing http headers and url for web scraping
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
url = "https://money.cnn.com/data/hotstocks/"

stock_symbl = {}
data = requests.get(url)
soup = BeautifulSoup(data.text, 'html.parser')

# Get live active tickers from https://money.cnn.com/data/hotstocks/
for i in range(1,11):
    name = list(soup.find('table', {'class':'wsod_dataTable wsod_dataTableBigAlt'}).find_all('tr')[i].text.split("\n"))[1]
    for i in range(len(name)):
        symbol, stock_name = name.split(' ', 1)
        stock_symbl[symbol] = stock_name
        
print(stock_symbl)


{'F': 'Ford Motor Co', 'CCL': 'Carnival Corp', 'T': 'AT&T Inc', 'BAC': 'Bank of America Corp', 'NCLH': 'Norwegian Cruise Line Holdings Ltd', 'PFE': 'Pfizer Inc', 'WFC': 'Wells Fargo & Co', 'DAL': 'Delta Air Lines Inc', 'C': 'Citigroup Inc', 'TWTR': 'Twitter Inc'}


In [3]:
#Scraping data from google finance website
#chromedrivers file need to be placed in the current working directory for selenium drivers
#to successfully execute scraping data from google finance website
stock_values = {}
openprice = 0
avg_volume = 0
pe_ratio = 0
stock_data = [[] for i in range(len(stock_symbl.keys()))]
cnt = 0
print(stock_data)

for symbol, stock_name in stock_symbl.items():
    google_url = f"https://www.google.com/finance/quote/{symbol}:NYSE"
    browser = webdriver.Chrome()
    browser.get(google_url)
    html_source = browser.page_source
    browser.quit()
    soup = BeautifulSoup(html_source, "lxml")
    for el in soup.find_all("div", {"class": "P6K39c"}):
        
        stock_data[cnt].append(el.get_text())
        
    cnt += 1
    
        
print(stock_data)

[[], [], [], [], [], [], [], [], [], []]
[['$19.14', '$18.61 - $19.49', '$8.43 - $20.79', '76.49B USD', '107.59M', '27.26', '2.08%', 'NYSE', 'A', 'Jim Farley', 'Jun 16, 1903', 'Dearborn, MichiganUnited States', 'ford.com', '186,000'], ['$17.20', '$17.40 - $19.19', '$16.32 - $31.52', '21.26B USD', '37.40M', '-', '-', 'NYSE', 'C', '1972', 'Miami, FloridaUnited States', 'carnivalcorp.com', '70,000'], ['$23.46', '$23.31 - $23.94', '$22.22 - $33.88', '166.56B USD', '48.89M', '196.77', '8.92%', 'NYSE', 'A-', 'John T. Stankey', 'Oct 5, 1983', 'Dallas, TexasUnited States', 'att.com', '230,000'], ['$43.87', '$43.96 - $44.88', '$28.14 - $48.69', '362.64B USD', '43.56M', '13.28', '1.90%', 'NYSE', 'A', 'Brian Moynihan', '1998', 'bankofamerica.com', '209,000'], ['$18.29', '$18.31 - $20.73', '$17.78 - $34.49', '8.42B USD', '21.45M', '-', '-', 'NYSE', 'B', 'Frank J. Del Rio', '1966', 'Miami, FloridaUnited States', 'nclhltd.com', '34,300'], ['$54.26', '$51.62 - $53.89', '$33.36 - $55.70', '304.61B USD

In [4]:
# combining active tickers with that corresponding open_price,avg_volume,pe_ratio values in the format of dictionary
for (symbol, stockdata) in zip(stock_symbl.keys(), stock_data):
    
    stock_values[symbol] = [stockdata[1].split("-")[0], stockdata[4], stockdata[5]]
        
print(stock_values)



{'F': ['$18.61 ', '107.59M', '27.26'], 'CCL': ['$17.40 ', '37.40M', '-'], 'T': ['$23.31 ', '48.89M', '196.77'], 'BAC': ['$43.96 ', '43.56M', '13.28'], 'NCLH': ['$18.31 ', '21.45M', '-'], 'PFE': ['$51.62 ', '43.15M', '15.34'], 'WFC': ['$48.74 ', '23.90M', '11.70'], 'DAL': ['$37.01 ', '14.17M', '-'], 'C': ['$62.14 ', '21.06M', '6.04'], 'TWTR': ['$41.75 ', '20.84M', '-']}


In [5]:
# Writing data to file

try:
    if os.path.exists("stocks.txt"):
        os.remove('stocks.txt')
    with open('stocks.txt', 'w') as f:
        f.write(f"Symbol\t OpenPrice\t Volume\t\t PeRatio\n")
        for key, value in stock_values.items():
            f.write(f"{key},\t {value[0]},\t {value[1]},\t {value[2]}\n")
        print("Writing file successful!")
except Exception as e:
    print("Error while writing file!")
    

Writing file successful!


In [6]:
# Inserting data into sqlite database

database = "./StocksDatabase.db"

try:
    connection = sql.connect(database, timeout=10)
    if connection:
        print("Connection to database successful!")
        
    cursor = connection.cursor()
    
    cursor.execute("CREATE TABLE IF NOT EXISTS STOCK (TickerSymbol TEXT NOT NULL, OpenPrice REAL, Volume INTEGER, PERatio REAL DEFAULT 0)")
    print(cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall())
    
    for key, value in stock_values.items():
        cursor.execute("INSERT INTO STOCK (TickerSymbol, OpenPrice, Volume, PERatio) VALUES (?,?,?,?)", (key, value[0], value[1], value[2]))
    connection.commit()
   
    cursor.execute("SELECT * FROM STOCK")
    print(cursor.fetchall())
    connection.close()
    
except sql.Error as e:
    print(e)
finally:
    connection.close()

Connection to database successful!
[('STOCK',)]
[('F', '$18.61 ', '107.59M', 27.49), ('CCL', '$17.40 ', '37.40M', '-'), ('T', '$23.33 ', '48.89M', 197.02), ('BAC', '$43.96 ', '43.56M', 13.33), ('NCLH', '$18.31 ', '21.45M', '-'), ('PFE', '$51.62 ', '43.15M', 15.33), ('WFC', '$48.74 ', '23.90M', 11.71), ('DAL', '$37.01 ', '14.17M', '-'), ('C', '$62.14 ', '21.06M', 6.06), ('TWTR', '$41.75 ', '20.84M', '-'), ('F', '$18.61 ', '107.59M', 27.26), ('CCL', '$17.40 ', '37.40M', '-'), ('T', '$23.31 ', '48.89M', 196.77), ('BAC', '$43.96 ', '43.56M', 13.28), ('NCLH', '$18.31 ', '21.45M', '-'), ('PFE', '$51.62 ', '43.15M', 15.34), ('WFC', '$48.74 ', '23.90M', 11.7), ('DAL', '$37.01 ', '14.17M', '-'), ('C', '$62.14 ', '21.06M', 6.04), ('TWTR', '$41.75 ', '20.84M', '-')]
